In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv(r"..\oblig3_og_4\student_performance.csv", delimiter=";")
df.info()

In [ ]:
df.describe()

In [ ]:

df.isna().sum()

In [ ]:

pd.set_option('display.max_columns', None)
df.head()

### Beskrivelse av dataset:
Datasettet består av 36 features/kolonner og en target/kolonne. Alle featurene er oppgitt i numeriske verdier, men flere har disse ser ut til å ha blitt transformert fra categoriske verdier/input og er ikke en del av en sammenhengende tallrekke. Hensikten med datasettet er å predikere akademisk frafall og si noe om hvilke features som påvirker dette. Hver rad i datasette representere en student.

- Feature kolonnene inneholder forskjellig informasjon om akademsike, demografiske og sosial økonomiske faktorer som var kjent ved oppstart.

- Target kolonnen oppgir 3 kategoriske verdier, Enrolled, Droppout og Graduated. Disse beskriver statusen studenten har ved avsluttning av normal vargihet for et fag.

- Datasette skal ikke inneholde noen manglende verdier.

- Det står også i beskrivelsen av datasette at det har blitt gjennomført en begtydelig preprossesering av datasettet for eventuelle uteliggere og manglende data.

In [ ]:
print(df['Target'].value_counts())
df['Target'].value_counts().plot.bar()

- Som vi ser av tellingen av kategoriene i target kolonnen, så er det noe skjevfordeling i antallet av de forskjellige kategoriene, men alle kategoriene er godt representert og ingen av katagoriene har et veldig lavt antall representasjoner.

In [ ]:
df_targets = pd.Series(LabelEncoder().fit_transform(df['Target']))
print(df_targets.value_counts())
df_targets.value_counts().plot.bar()

- Siden target kollonnen i utgangspunktet inneholdeholder kategoriskeverdier så må vi transformere disse før vi bruker datasette videre.

In [ ]:
df_features = df.drop("Target", axis=1)
df_features.hist(figsize=(20,20))
plt.show()

#### Histogram av feature kolonnene.
- Histogrammet viser at det er betydelig forskjell i skalaene til de forskjellige featurene og det vil derfor være hensiktsmessig å gjennomføre en scalering slik at disse er repsresetert innenfor samme scalering. Jeg kommer til å bruke en standard mean scalering til å gjennomføre dette.

- Vi ser også at datasettet inneholder en kobinasjon av kolonner med kontinuelige tallrekker og transformerte kategoriske data. Vi ser også at mange av kolonnen inneholder data som ikke er normalfordelt, men det er i utgangspunkt vansklig å gjøre noe med dette ut i fra sammensettning av datasetet uten å potensielt miste mye informasjon, det vil derfor være hensiksmessig å bruke modeller som ikke er avhenig av normalfordelete data for å prestere bra. 

In [ ]:
df_features = pd.DataFrame(StandardScaler().fit_transform(df_features), columns=df_features.columns)
df_features.hist(figsize=(20,20))
plt.show()

In [ ]:
df_enrolled = df[df['Target'] == 'Enrolled']
df_dropout = df[df['Target'] == 'Dropout'].sample(len(df_enrolled))
df_graduate = df[df['Target'] == 'Graduate'].sample(len(df_enrolled))

df_downsample = pd.concat([df_enrolled, df_dropout, df_graduate])
df_downsample = df_downsample.sample(frac=1)

df_downsample_features = pd.DataFrame(StandardScaler().fit_transform(df_downsample.drop('Target', axis=1)), columns=df_downsample.drop('Target', axis=1).columns)
df_downsample_target = pd.Series(LabelEncoder().fit_transform(df_downsample['Target']))

In [ ]:
print(df_downsample_target.value_counts())
df_downsample_target.value_counts().plot.bar()

In [ ]:
print(cross_val_score(RandomForestClassifier(), X=df_downsample_features, y=df_downsample_target))
print(cross_val_score(RandomForestClassifier(), X=df_features, y=df_targets))

In [ ]:
print(cross_val_score(GradientBoostingClassifier(), X=df_downsample_features, y=df_downsample_target))
print(cross_val_score(GradientBoostingClassifier(), X=df_features, y=df_targets))

In [ ]:
print(cross_val_score(LogisticRegression(), X=df_downsample_features, y=df_downsample_target))
print(cross_val_score(LogisticRegression(), X=df_features, y=df_targets))

#### Test av downsampling på target kolonnen
- Selvom alle kategoriene i target kolonnen har en god representasjon av verdier, så har jeg valg å gjennomføre en test av downsampling på datasettet for å se om dette kan gi noen positiv påvirking på treningen av datasettet.

- For å gjennomføre downsamplingen har jeg downsamplet katagoriene for "Graduated" og "Droppout" mot antallet av verdier i "Enrolled", slik at disse er likt representert i datasettet. Deretter har jeg brukt cross_val_score til å gjennomføre en enkel test for å sammenligne downsampling og ikke for de modellene jeg planlegger å bruke videre i analysen.

- Som vi kan se av resultatene i cellene over ga det ikke bedre resultater å gjennomføre en downsampling, så jeg kommer ikke til å bruke dette videre i analysen.

In [ ]:
df_correlation = df_features.corr()
plt.figure(figsize=(20,20))
sb.heatmap(df_correlation, annot=True, cmap="coolwarm", fmt=".2f")

#### Korrelasjon
- Som vi kan se av plotet over er det noen features som har en betydelig korrelasjon, jeg kommer defor til bruke PCA til å teste om noen eller flere av featurene kan kombineres for å redusere denne korrelasjonen, samt gjøre datasettet mindre og mindre tidskrevende å trene, tune og teste.

In [ ]:
features_pca10 = pd.DataFrame(PCA(n_components=10).fit_transform(df_features))
features_pca14 = pd.DataFrame(PCA(n_components=14).fit_transform(df_features))
features_pca18 = pd.DataFrame(PCA(n_components=18).fit_transform(df_features))
features_pca22 = pd.DataFrame(PCA(n_components=22).fit_transform(df_features))
features_pca26 = pd.DataFrame(PCA(n_components=26).fit_transform(df_features))
features_pca29 = pd.DataFrame(PCA(n_components=29).fit_transform(df_features))
features_pca32 = pd.DataFrame(PCA(n_components=32).fit_transform(df_features))
features_pca35 = pd.DataFrame(PCA(n_components=35).fit_transform(df_features))

In [ ]:
print(cross_val_score(RandomForestClassifier(), X=features_pca10, y=df_targets))
print(cross_val_score(RandomForestClassifier(), X=features_pca14, y=df_targets))
print(cross_val_score(RandomForestClassifier(), X=features_pca18, y=df_targets))
print(cross_val_score(RandomForestClassifier(), X=features_pca22, y=df_targets))
print(cross_val_score(RandomForestClassifier(), X=features_pca26, y=df_targets))
print(cross_val_score(RandomForestClassifier(), X=features_pca29, y=df_targets))
print(cross_val_score(RandomForestClassifier(), X=features_pca32, y=df_targets))
print(cross_val_score(RandomForestClassifier(), X=features_pca35, y=df_targets))
print(cross_val_score(RandomForestClassifier(), X=df_features, y=df_targets))


In [ ]:
print(cross_val_score(GradientBoostingClassifier(), X=features_pca10, y=df_targets))
print(cross_val_score(GradientBoostingClassifier(), X=features_pca14, y=df_targets))
print(cross_val_score(GradientBoostingClassifier(), X=features_pca18, y=df_targets))
print(cross_val_score(GradientBoostingClassifier(), X=features_pca22, y=df_targets))
print(cross_val_score(GradientBoostingClassifier(), X=features_pca26, y=df_targets))
print(cross_val_score(GradientBoostingClassifier(), X=features_pca29, y=df_targets))
print(cross_val_score(GradientBoostingClassifier(), X=features_pca32, y=df_targets))
print(cross_val_score(GradientBoostingClassifier(), X=features_pca35, y=df_targets))
print(cross_val_score(GradientBoostingClassifier(), X=df_features, y=df_targets))

In [ ]:
print(cross_val_score(LogisticRegression(), X=features_pca10, y=df_targets))
print(cross_val_score(LogisticRegression(), X=features_pca14, y=df_targets))
print(cross_val_score(LogisticRegression(), X=features_pca18, y=df_targets))
print(cross_val_score(LogisticRegression(), X=features_pca22, y=df_targets))
print(cross_val_score(LogisticRegression(), X=features_pca26, y=df_targets))
print(cross_val_score(LogisticRegression(), X=features_pca29, y=df_targets))
print(cross_val_score(LogisticRegression(), X=features_pca32, y=df_targets))
print(cross_val_score(LogisticRegression(), X=features_pca35, y=df_targets))
print(cross_val_score(LogisticRegression(), X=df_features, y=df_targets))

#### PCA testing
- For å test ut som PCA kan gi bedre resultater for treningen av datasettet så har jeg bruket PCA til å samle datasette til ferre features, jeg valgt å test dette over en range (14-35, samt uendrett) for å se om gir noen positiv effekt på prestajonen for modellen på dette datasettet.

- Akkurat som ved testing av downsampling har jeg bruk cross_val_score til å gjennoføre en enkelt testing for de modellene jeg har planer om å bruke videre.

- Ut i fra resultatene som denne testingen gir, så ser det ikke ut til å være hensiktsmessig å bruke PCA på dette datasette videre i analysen i forhold til prestasjonen til datasettet da det ikke gir noen betydelig bedre resultater for noen av modellene. Vi ser også av resultatene at prestasjonene til datasettet generell er bedre desto flere av featurene som er bevart.

- For LogisticRegession er det inmidlertid potensielt noe bedre resultat for en PCA kombinasjon med 32 features og hvis vi kun skulle ha jobbet videre med denne modellen vil dette vært hensikts messig å undersøke videre med videre tuneing.